### Importing files

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import AutoTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/mlba/train.csv")
df_train.head()

,Sequence,Label
0,MTAELRNLPHIASMAFNEPLMLEPAYARVFFCALAGQLGISSLTDA...,1
1,MTDITANVVVSNPRPIFTESRSFKAVANGKIYIGQIDTDPVNPANQ...,1
2,MEMISNNLNWFVGVVEDRMDPLKLGRVRVRVVGLHPPQRAQGDVMG...,1
3,MIINLADVEQLSIKAESVDFQYDMYKKVCEKFTDFEQSVLWQCMEA...,1
4,MRAFSTLDRENETFVPSVRVYADGETEDNSFSLKYRSNWTPGRFNS...,1


### Pre-processing Data

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(df_train["Sequence"], df_train["Label"].values, shuffle = True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

integer_encoder = LabelEncoder()
integer_encoder.fit(list('ACDEFGHIKLMNPQRSTVWY'))
X_integer_encoded = [integer_encoder.transform(list(seq)) for seq in x_train]
X_val_encoded = [integer_encoder.transform(list(seq)) for seq in x_val]

x_integer_encoded_pad = tf.keras.preprocessing.sequence.pad_sequences(X_integer_encoded)
x_val_encoded_pad = tf.keras.preprocessing.sequence.pad_sequences(X_val_encoded)

In [ ]:
def pad_enc(x):
  integer_encoder = LabelEncoder()
  integer_encoder.fit(list('ACDEFGHIKLMNPQRSTVWY'))
  x = [integer_encoder.transform(list(seq)) for seq in x]
  x = tf.keras.preprocessing.sequence.pad_sequences(x)
  return x

### Making CNN

In [ ]:
vocab_size = 21

In [ ]:
# Variable-length int sequences.
Input = tf.keras.Input(shape=(None, ), dtype='float')
# Embedding lookup.
token_embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=16)
# Query embeddings of shape [batch_size, Tq, dimension].
query_embeddings = token_embedding(Input)
# Value embeddings of shape [batch_size, Tv, dimension].
value_embeddings = token_embedding(Input)

# CNN layer.
cnn_layer = tf.keras.layers.Conv1D(
    filters=100,
    kernel_size=2,
    # Use 'same' padding so outputs have the same shape as inputs.
    padding='same')
# Query encoding of shape [batch_size, Tq, filters].
query_seq_encoding = cnn_layer(query_embeddings)
# Value encoding of shape [batch_size, Tv, filters].
value_seq_encoding = cnn_layer(value_embeddings)

# Query-value attention of shape [batch_size, Tq, filters].
query_value_attention_seq = tf.keras.layers.Attention()(
    [query_seq_encoding, value_seq_encoding])

# Reduce over the sequence axis to produce encodings of shape
# [batch_size, filters].
query_encoding = tf.keras.layers.GlobalAveragePooling1D()(
    query_seq_encoding)
query_value_attention = tf.keras.layers.GlobalAveragePooling1D()(
    query_value_attention_seq)

# Concatenate query and document encodings to produce a DNN input layer.
input_layer = tf.keras.layers.Concatenate()(
    [query_encoding, query_value_attention])

x = tf.keras.layers.Dense(128, activation = "relu")(input_layer)
x = tf.keras.layers.Dense(64, activation = "relu")(x)
x = tf.keras.layers.Dense(32, activation = "relu")(x)
x = tf.keras.layers.Dense(16, activation = "relu")(x)
output = tf.keras.layers.Dense(1, activation = "sigmoid") (x)

model = tf.keras.Model(inputs= Input, outputs = output)
model.compile(optimizer = "adam", metrics = ["accuracy"], loss = "binary_crossentropy")
model.summary()


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_13 (Embedding)    (None, None, 16)             320       ['input_13[0][0]',            
                                                                     'input_13[0][0]']            
                                                                                                  
 conv1d_39 (Conv1D)          (None, None, 100)            3300      ['embedding_13[0][0]',        
                                                                     'embedding_13[1][0]']        
                                                                                           

### Training the model

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor = "val_loss")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

In [ ]:
model.fit(x_integer_encoded_pad, y_train, validation_data= (x_val_encoded_pad, y_val), callbacks = [early_stopping, reduce_lr], epochs = 60)

### Refitting the model on the entire dataset after reinitializing

In [ ]:
x = pad_enc(df_train["Sequence"])

In [ ]:
x

array([[ 0,  0,  0, ..., 16, 12, 17],
       [ 0,  0,  0, ...,  0, 16,  9],
       [ 0,  0,  0, ...,  2,  7,  5],
       ...,
       [ 0,  0,  0, ...,  5,  0,  3],
       [ 0,  0,  0, ...,  2,  2,  5],
       [ 0,  0,  0, ...,  2,  2,  3]], dtype=int32)

In [ ]:
model.fit(x, df_train["Label"].values, epochs = 40)

Epoch 1/40
48/48 [==============================] - 14s 225ms/step - loss: 0.5861 - accuracy: 0.7444
Epoch 2/40
48/48 [==============================] - 11s 228ms/step - loss: 0.5568 - accuracy: 0.7444
Epoch 3/40
48/48 [==============================] - 11s 230ms/step - loss: 0.5408 - accuracy: 0.7523
Epoch 4/40
48/48 [==============================] - 11s 231ms/step - loss: 0.5134 - accuracy: 0.7497
Epoch 5/40
48/48 [==============================] - 11s 233ms/step - loss: 0.5052 - accuracy: 0.7503
Epoch 6/40
48/48 [==============================] - 11s 235ms/step - loss: 0.5011 - accuracy: 0.7589
Epoch 7/40
48/48 [==============================] - 11s 238ms/step - loss: 0.4929 - accuracy: 0.7602
Epoch 8/40
48/48 [==============================] - 11s 239ms/step - loss: 0.4926 - accuracy: 0.7688
Epoch 9/40
48/48 [==============================] - 11s 240ms/step - loss: 0.4937 - accuracy: 0.7781
Epoch 10/40
48/48 [==============================] - 12s 240ms/step - loss: 0.4645 - accura

### Getting test predictions

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/mlba/test.csv")
df_test.head()

,ID,Sequence
0,501,ASNFTQFVLVNDGGTGNVTVAPSNFANGVAEWISSNSRSQAYKVTC...
1,502,ASNFTQFVLVNDGGTGNVTVAPSNFANGVAEWISSNSRSQAYKVTC...
2,503,MAATHTLPLASPGMARICLYGDLQRFGRRIDLRVKTGAEAIRALAT...
3,504,MADLKVGSTTGGSVIWHQGNFPLNPAGDDVLYKSFKIYSEYNKPQA...
4,505,MADLKVGSTTGGSVIWHQGNFPLNPAGDDVLYKSFKIYSEYNKPQA...


In [ ]:
x_test = df_test["Sequence"]
integer_encoder = LabelEncoder()
integer_encoder.fit(list('ACDEFGHIKLMNPQRSTVWY'))
X_test_encoded = [integer_encoder.transform(list(seq)) for seq in x_test]

x_test_encoded_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_encoded)

In [ ]:
x_test_encoded_pad.shape

(310, 1289)

In [ ]:
preds = model.predict(x_test_encoded_pad).flatten()

10/10 [==============================] - 0s 17ms/step


In [ ]:
preds = [i[1] for i in y_pred]
preds = np.array(preds)

### Saving as CSV

In [ ]:
final = {"ID":df_test["ID"].values, "Label": preds}
final_df = pd.DataFrame(final)
final_df.head()

In [ ]:
final_df.to_csv("/content/drive/MyDrive/mlba/gruop80_predictions_output.csv", index = False)